In [ ]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#PromptTemplate: string을 받고 / ChatPromptTemplate은 messages를 받는다.

# Outparser -> response의 형태를 변형하고 싶을 때 (tuple, list 등)

# llm = OpenAI()
chat = ChatOpenAI(temperature=0.1,streaming=True) # 모델의 창의성의 정도. 0.1은 극히 낮은 수준으로 비용 절감 가능

In [59]:
# from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages([
    ("system","You are a cook expert. And you only reply in {country}"),
    ("ai","Hola. ¡Mi nombre es {name}!"),
    ("human","What is the recipe of {cuisine}. Also, What is your name?")
])

# chat.predict_messages(messages)

prompt = template.format_messages(
    country="Japaness",
    name = "Nakamoto",
    cuisine = "Sushi"
)

chat.predict_messages(prompt)




AIMessage(content='こんにちは！寿司のレシピは以下の通りです：\n\n材料：\n- しゃり（酢飯）：米、酢、砂糖、塩\n- ネタ：新鮮な魚、野菜、卵焼き、海苔など\n- しょうゆ、わさび、生姜\n\n作り方：\n1. 米を研いで水につけ、炊飯器で炊く。\n2. 炊き上がったご飯に酢、砂糖、塩を加えて混ぜる。\n3. 魚や野菜を切って準備する。\n4. 手に水をつけて、しゃりを握り、ネタをのせる。\n5. しょうゆ、わさび、生姜を添えて完成！\n\n私の名前は中本です。どうぞよろしくお願いします！')

In [60]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser) :
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

p = CommaOutputParser()

p.parse("Hello  ,how,are,you")

['Hello', 'how', 'are', 'you']

In [61]:
template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are aksed will be answered with a comma separatedß list of max {max_itmes}. Do not reply with anything else."),
    ("human","{question}"),
])

prompt = template.format_messages(
    max_itmes=10,
    question = "what are the planet?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)




['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [62]:
from code import compile_command


template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are aksed will be answered with a comma separatedß list of max {max_itmes}. Do not reply with anything else."),
    ("human","{question}"),
])

prompt = template.format_messages(
    max_itmes=10,
    question = "what are the planet?"
)

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_itmes" : 5,
    "question" : "What is your strength?"
})



['Efficiency', 'Accuracy', 'Consistency', 'Speed', 'Adaptability']

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature = 0.1)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients"),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 


In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipes vegitarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
   "cuisine" : "indian"
})


As a vegetarian chef, I can help you make a delicious vegetarian version of Chicken Tikka Masala. Here are the alternative ingredients and preparation methods:

1. **Alternative to Chicken**: Replace the chicken with firm tofu or paneer (Indian cottage cheese) cut into bite-sized pieces. Press the tofu to remove excess water, then marinate it in the yogurt and spice mixture as you would with the chicken. For paneer, you can skip the marination step as it absorbs flavors well during cooking.

2. **Alternative to Heavy Cream**: You can use coconut cream or cashew cream as a dairy-free alternative to heavy cream. To make cashew cream, soak raw cashews in water for a few hours, then blend them with water until smooth. Add the cashew cream towards the end of cooking to achieve a creamy texture.

3. **Alternative to Yogurt**: Use dairy-free yogurt made from soy, almond, or coconut milk as a substitute for plain yogurt in the marinade. Make sure it has a similar consistency to yogurt for the 

AIMessageChunk(content="As a vegetarian chef, I can help you make a delicious vegetarian version of Chicken Tikka Masala. Here are the alternative ingredients and preparation methods:\n\n1. **Alternative to Chicken**: Replace the chicken with firm tofu or paneer (Indian cottage cheese) cut into bite-sized pieces. Press the tofu to remove excess water, then marinate it in the yogurt and spice mixture as you would with the chicken. For paneer, you can skip the marination step as it absorbs flavors well during cooking.\n\n2. **Alternative to Heavy Cream**: You can use coconut cream or cashew cream as a dairy-free alternative to heavy cream. To make cashew cream, soak raw cashews in water for a few hours, then blend them with water until smooth. Add the cashew cream towards the end of cooking to achieve a creamy texture.\n\n3. **Alternative to Yogurt**: Use dairy-free yogurt made from soy, almond, or coconut milk as a substitute for plain yogurt in the marinade. Make sure it has a similar 

In [ ]:
from langchain.chat_models import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# example_template = """
#     Human : {question}
#     AI : {answer}
# """

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country" : "Vietnam"
})

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")


AI:
I know this:
Capital: Hanoi
Language: Vietnamese
Food: Pho and Banh Mi
Currency: Vietnamese Dong

AIMessageChunk(content='AI:\nI know this:\nCapital: Hanoi\nLanguage: Vietnamese\nFood: Pho and Banh Mi\nCurrency: Vietnamese Dong')

In [83]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"country": "France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# example_template = """
#     Human : {question}
#     AI : {answer}
# """

# example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")


# AI야 너 이렇게 질문이 들어오면 저렇게 답변 했었어! 라고 주입 교육 
example_prompt = ChatPromptTemplate.from_messages([
    ("human","What do you know about {country}?"),
    ("ai","{answer}")
])

# prompt = FewShotPromptTemplate(
#     example_prompt = example_prompt,
#     examples = examples,
#     suffix = "Human : What do you know about {country}?",
#     input_variables=["country"]
# )

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert, you have to add the information of geography"),
    example_prompt,
    ("human","What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country" : "Korea"
})

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")


I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='I know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"movie": "The Godfather?",
"answer": """
Here is what I know:
Director: Francis Ford Coppola
Primary Actors : Marlon Brando, Al Pacino, James Caan, Robert Duvall
Budget: Around $6 million
Profit: Around $250 million (Global Box Office)
Genre: Crime, Drama
Short Synopsis: Centered on the Corleone family, an Italian-American mafia dynasty, the film chronicles the transfer of power from the aging patriarch Vito Corleone (Marlon Brando) to his reluctant son Michael Corleone (Al Pacino). As Michael is drawn deeper into the ruthless world of organized crime, he transforms into the new Godfather, marking one of the greatest character arcs in cinema history.
""",
},
{
"movie": "Titanic?",
"answer": """
Here is what I know:
Director: James Cameron
Primary Actors : Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates
Budget: Around $200 million
Profit: Around $2.2 billion (Global Box Office, ranked 4th all-time)
Genre: Romance, Drama, Disaster
Short Synopsis: Based on a true story, this film blends romance and disaster elements. Set in 1912, it follows the ill-fated maiden voyage of the luxury liner Titanic, which strikes an iceberg and sinks. The story revolves around the passionate love affair between Jack (Leonardo DiCaprio) and Rose (Kate Winslet), who come from different social classes but defy all odds to be together.
""",
},
{
"movie": "The Shawshank Redemption?",
"answer": """
I know this:
Here is what I know:
Director: Frank Darabont
Primary Actors: Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler
Budget: Around $25 million
Profit: Around $73 million (Box Office + Home Video Sales)
Genre: Drama, Prison Film
Short Synopsis:After being wrongfully convicted and sentenced to life in Shawshank Prison, Andy Dufresne (Tim Robbins) befriends Red (Morgan Freeman) and navigates life behind bars with intelligence and hope. The film explores themes of friendship, injustice, and the enduring power of hope, leading to an unexpected and brilliant twist.
""",
},
]

# AI야 너 이렇게 질문이 들어오면 저렇게 답변 했었어! 라고 주입 교육 
example_prompt = ChatPromptTemplate.from_messages([
    ("human","What do you know about {movie}?"),
    ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a movie expert"),
    example_prompt,
    ("human","What do you know about {movie}?")
])

chain = final_prompt | chat

chain.invoke({
    "movie" : "Avengers"
})




Here is what I know:
Director: Pete Docter
Primary Actors: Amy Poehler, Phyllis Smith, Bill Hader, Lewis Black, Mindy Kaling
Budget: Around $175 million
Profit: Around $858 million (Global Box Office)
Genre: Animation, Comedy, Drama
Short Synopsis: Inside Out is an animated film that takes place inside the mind of an 11-year-old girl named Riley. The film personifies her emotions - Joy, Sadness, Anger, Fear, and Disgust - as characters who guide her through life's challenges. As Riley navigates a difficult time in her life, the emotions must work together to help her find balance and happiness. The film beautifully explores the complexities of emotions and the importance of embracing all feelings.

AIMessageChunk(content="Here is what I know:\nDirector: Pete Docter\nPrimary Actors: Amy Poehler, Phyllis Smith, Bill Hader, Lewis Black, Mindy Kaling\nBudget: Around $175 million\nProfit: Around $858 million (Global Box Office)\nGenre: Animation, Comedy, Drama\nShort Synopsis: Inside Out is an animated film that takes place inside the mind of an 11-year-old girl named Riley. The film personifies her emotions - Joy, Sadness, Anger, Fear, and Disgust - as characters who guide her through life's challenges. As Riley navigates a difficult time in her life, the emotions must work together to help her find balance and happiness. The film beautifully explores the complexities of emotions and the importance of embracing all feelings.")